In [8]:
%pip install torch
%pip install torchvision

     ---------------------------------------- 1.1/1.1 MB 5.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 23.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [98]:
%pip install --upgrade numpy

     ---------------------------------------- 14.8/14.8 MB 3.8 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.4
    Uninstalling numpy-1.21.4:
      Successfully uninstalled numpy-1.21.4
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not install packages due to an OSError: [WinError 5] Accès refusé: 'C:\\Users\\pierr\\AppData\\Local\\Programs\\Python\\Python310\\Lib\\site-packages\\~umpy\\.libs\\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll'
Consider using the `--user` option or check the permissions.


[notice] A new release of pip available: 22.3 -> 23.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:

import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path

from PIL import Image
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import numpy as np
import torch
import torchvision.transforms as transforms
import torchvision.models as models
from skimage import io
import os

c:\Users\pierr\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [30]:
pathToCategories = "./baseline/baseline/"
labels=["highway", "office", "pedestrians", "PETS2006"]
# fonction d'acquisition des donnees dentrees et de validation pour l'analyse par rapport a une categories de video mis en parametre
def init_dataset(labels):
    input_data = {}
    groundTruth_data = {}
    i=0
    for label in labels:
        input_data[label] = [str(p) for p in Path(
            pathToCategories+label+"/input/").glob('*.jpg')]
        groundTruth_data[label] = [str(p) for p in Path(
            pathToCategories+label+"/groundtruth/").glob('*.png')]
        
        #if(i > batch):
        #    break
        #i+=1
        
    return input_data,groundTruth_data


In [31]:
input_data,groundTruth_data=init_dataset(labels,4)

TypeError: init_dataset() takes 1 positional argument but 2 were given

### Segmentation sémantique

In [32]:
#fonction appliquant des pretransformation aux cathegories dimage recu et retournant la liste des images retransforme en Tensor

preprocessing = transforms.Compose([
    transforms.ToTensor(),
])


nb_img_batch=4
def get_input_tensor(labels):
    input_tensor_label = {}
    for label in labels:
        input_tensor_label[label] = []
        for input in input_data[label]:
            tensor = preprocessing(Image.fromarray(io.imread(input)))#io.imread(os.path.join(input)) )
            #tensor = preprocessing(cv2.imread("./"+input))
            input_tensor_label[label].append(tensor)
    return input_tensor_label


In [26]:
input_tensor_label=get_input_tensor(labels)

KeyboardInterrupt: 

In [33]:
def stack_input_tensor(labels,input_tensor_label):
    input_batch_tensor_label = {}
    for label in labels:
        #input_batch_tensor_label[label] =[]
        #for tensor in input_tensor_label[label]:
        input_batch_tensor_label[label]=torch.stack(input_tensor_label[label], 0)
        #input_batch_tensor_label[label] = input_batch_tensor_label[label].cuda() 
    return input_batch_tensor_label


In [10]:
model = models.segmentation.deeplabv3_resnet101(pretrained=True).eval()

with torch.no_grad():
    predictions = model(input_batch_tensor_label)


c:\Users\pierr\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\pierr\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DeepLabV3_ResNet101_Weights.COCO_WITH_VOC_LABELS_V1`. You can also use `weights=DeepLabV3_ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/deeplabv3_resnet101_coco-586e9e4e.pth" to C:\Users\pierr/.cache\torch\hub\checkpoints\deeplabv3_resnet101_coco-586e9e4e.pth
  8%|▊         | 18.7M/233M [00:04<00:45, 4.89MB/s]


KeyboardInterrupt: 

## Fonction Main de lexecution

In [34]:
labels = ["highway", "office", "pedestrians", "PETS2006"]
batch = 4
mini_input_data={}
for label in labels:
    mini_input_data[label]=input_data[label][:4]

input_data, true_data = init_dataset(labels)
mini_input_data={}
for label in labels:
    mini_input_data[label]=input_data[label][:4]
input_tensor_label = get_input_tensor(mini_input_data)
input_batch_tensor_label = stack_input_tensor(input_tensor_label)

